## 使用Function_call模式

> 注意：Prompt中的 agent_scratchpad 参数必须填写

In [2]:
import os
import dotenv
from langchain.agents import create_tool_calling_agent, AgentExecutor, create_react_agent
from langchain_community.tools import TavilySearchResults
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

# 搜索工具
search = TavilySearchResults(max_results=5)

# 自定义提示词模版
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个乐于助人的助手，请务必使用tavily_search_results_json 工具获取信息"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# 定义LLM
llm = ChatOpenAI(
    model_name="gpt-4o-mini"
)
# 创建Agent对象
agent = create_tool_calling_agent(
    llm=llm,
    tools=[search],
    prompt=prompt_template
)
# 创建AgentExecutor执行器
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search],
    verbose=True
)
# 测试
result = agent_executor.invoke({"input": "今天北京的天气怎么样？"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '北京天气'}`


SSLError(MaxRetryError("HTTPSConnectionPool(host='api.tavily.com', port=443): Max retries exceeded with url: /search (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1016)')))"))
Invoking: `tavily_search_results_json` with `{'query': 'Beijing weather today'}`


[{'title': 'Beijing October 2025 Historical Weather Data (China)', 'url': 'https://weatherspark.com/h/m/131055/2025/10/Historical-Weather-in-October-2025-in-Beijing-China', 'content': '| Today  Yesterday  Oct  2025  194019501960197019801990200020102020  2016201720182019202020212022202320242025  SpringSummerFallWinter  JanFebMarAprMayJunJulAugSepOctNovDec  12345678910111213141516171819202122232425262728293031 October 2025 Weather History in Beijing China The data for this report comes from the Beijing Capital International Airport. See all nearby weather st

## ReAct 模式

In [6]:
from langchain import hub
from langchain.agents.react.agent import create_react_agent
tools = [TavilySearchResults(max_results=5)]

# 远程获取模版， 其中提供了默认的提示词模版
template = hub.pull("hwchase17/react")

llm = ChatOpenAI(
    model_name="gpt-4o-mini"
)

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=template
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools,
    verbose=True, handle_parsing_errors=True
)

agent_executor.invoke({"input": "今天济南章丘的天气怎么样？"})

/opt/anaconda3/envs/py311/lib/python3.11/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
我需要查找关于济南章丘今天的天气信息。  
Action: tavily_search_results_json  
Action Input: "今天 济南 章丘 天气"  [{'title': '章丘天气预报,章丘7天天气预报,章丘15天天气预报,章丘天气查询', 'url': 'https://www.weather.com.cn/weather/101120104.shtml?', 'content': '15日（今天）. 小雨转晴. 20/12℃. <3级 · 16日（明天）. 小雨. 20/13℃. <3级 · 17日（后天）. 中雨转小雨. 15/11℃. <3级 · 18日（周六）. 多云. 14/6℃. <3级 · 19日（周日）. 多云. 14/5℃.', 'score': 0.98501}, {'title': '章丘天气预报,章丘7天天气预报,章丘15天天气预报,章丘天气查询', 'url': 'https://www.weather.com.cn/weather/101120104.shtml', 'content': '14日（今天）. 多云转阴. 22/15℃. <3级 · 15日（明天）. 小雨转多云. 20/14℃. <3级 · 16日（后天）. 多云转小雨. 21/14℃. <3级 · 17日（周五）. 小雨. 16/8℃. <3级 · 18日（周六）. 晴.', 'score': 0.98216}, {'title': '章丘 - 中国气象局-天气预报-城市预报', 'url': 'http://weather.cma.cn/web/weather/54727.html', 'content': '星期三 10/15. 小雨 · 20℃ ; 星期四 10/16. 小雨 · 20℃ ; 星期五 10/17. 中雨 · 15℃ ; 星期六 10/18. 多云 · 14℃ ; 星期日 10/19. 多云 · 14℃.', 'score': 0.98164}, {'title': '章丘-天气预报 - 中央气象台', 'url': 'https://www.nmc.cn/publish/forecast/ASD/zhangqiu.html', 'conten

{'input': '今天济南章丘的天气怎么样？', 'output': '今天济南章丘的天气是小雨转晴，气温20℃/12℃。'}

In [21]:
from langchain.agents.react.agent import create_react_agent
tools = [TavilySearchResults(max_results=5)]

# 提示词模版
template_str = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
"""
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", template_str),
    ("system", "Thought:{agent_scratchpad}"),
    ("human", "我的问题是：{input}"),
])

llm = ChatOpenAI(
    model_name="gpt-4o-mini"
)

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=chat_prompt_template
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools,
    verbose=True, handle_parsing_errors=True,
    max_iterations=6
)

result = agent_executor.invoke({"input": "今天深圳的天气怎么样？"})

print(result)



> Entering new AgentExecutor chain...
Thought: 我需要查找今天深圳的天气情况。  
Action: tavily_search_results_json  
Action Input: 深圳今天的天气  [{'title': '深圳 - 中国气象局-天气预报-城市预报', 'url': 'https://weather.cma.cn/web/weather/59493.html', 'content': '|  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |\n| 气温 | 30.8℃ | 30.9℃ | 29.5℃ | 30.4℃ | 27.9℃ | 26.1℃ | 25.5℃ | 25.6℃ |\n| 降水 | 6.4mm | 0.9mm | 0.6mm | 1.7mm | 0.3mm | 无降水 | 无降水 | 无降水 |\n| 风速 | 3.3m/s | 3.3m/s | 3.2m/s | 2.6m/s | 2.6m/s | 3.2m/s | 3.2m/s | 3m/s |\n| 风向 | 东北风 | 东北风 | 东南风 | 东南风 | 东南风 | 东北风 | 东北风 | 东北风 |\n| 气压 | 1008hPa | 1007.8hPa | 1007.5hPa | 1007.9hPa | 1008.4hPa | 1008.6hPa | 1008.8hPa | 1009.3hPa | [...] |  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 | 08:00 |\n| 天气 |\n| 气温 | 31.6℃ | 33.7℃ | 31.5℃ | 29.2℃ | 28.4℃ | 27.4℃ | 26.7℃ | 27.7℃ |\n| 降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 |\